Import sparksession module

In [23]:
from pyspark.sql import SparkSession

create spark session and include all the configuration needed

In [24]:
spark = SparkSession.builder.appName("epidermic_warehouse")\
        .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,mysql:mysql-connector-java:8.0.32')\
        .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
        .config('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog')\
        .config('spark.sql.catalog.spark_catalog.type', 'hive')\
        .config('spark.sql.catalog.local', 'org.apache.iceberg.spark.SparkCatalog')\
        .config('spark.sql.catalog.local.type','hadoop')\
        .config('spark.sql.catalog.local.warehouse','epidermic_warehouse')\
        .config('spark.sql.defaultCatalog','local')\
        .getOrCreate()
sqlContext = SparkSession(spark)
spark.sparkContext.setLogLevel("ERROR")

View the created spark session

In [25]:
spark

Read data in the mysQL that is stored in the dengue_features_test

In [5]:
sql_df = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://192.168.0.104:3306/erick") \
    .option("dbtable", "dengue_features_test") \
    .option("user", "root") \
    .option("password", "erick") \
    .load()

view the schema

In [6]:
sql_df.printSchema()

root
 |-- city: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- week_start_date: string (nullable = true)
 |-- ndvi_ne: string (nullable = true)
 |-- ndvi_nw: string (nullable = true)
 |-- ndvi_se: double (nullable = true)
 |-- ndvi_sw: double (nullable = true)
 |-- precipitation_amt_mm: double (nullable = true)
 |-- reanalysis_air_temp_k: double (nullable = true)
 |-- reanalysis_avg_temp_k: double (nullable = true)
 |-- reanalysis_dew_point_temp_k: double (nullable = true)
 |-- reanalysis_max_air_temp_k: double (nullable = true)
 |-- reanalysis_min_air_temp_k: double (nullable = true)
 |-- reanalysis_precip_amt_kg_per_m2: double (nullable = true)
 |-- reanalysis_relative_humidity_percent: double (nullable = true)
 |-- reanalysis_sat_precip_amt_mm: double (nullable = true)
 |-- reanalysis_specific_humidity_g_per_kg: double (nullable = true)
 |-- reanalysis_tdtr_k: double (nullable = true)
 |-- station_avg_temp_c: double (nul

we can use pandas to view our data because the dataset has soo many columns to fit in the window

In [26]:
import pandas as pd
pd.DataFrame(sql_df.take(5), columns=sql_df.columns).head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.0189,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.018,-0.0124,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,,-0.01986667,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.03983333,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


create HIVE Tempview to be able to carry out SQL operations

In [27]:
sql_df.createOrReplaceTempView("epidemic_prediction")

In [28]:
my_df=sqlContext.sql("SELECT * from epidemic_prediction")
pd.DataFrame(my_df.take(10), columns=my_df.columns).head(20)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.0189,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.018,-0.0124,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,,-0.01986667,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.03983333,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1
5,sj,2008,23,2008-06-03,-0.044,-0.03046667,0.132000,0.083529,71.17,299.768571,...,22.55,76.557143,71.17,16.667143,2.857143,28.000000,5.171429,32.8,25.0,27.7
6,sj,2008,24,2008-06-10,-0.0443,-0.024925,0.132271,0.159157,48.99,300.062857,...,13.10,76.844286,48.99,17.010000,3.157143,27.400000,6.042857,31.1,23.3,91.7
7,sj,2008,25,2008-06-17,,0.08215,0.144371,0.116729,30.81,300.484286,...,7.20,76.870000,30.81,17.420000,3.900000,28.757143,6.985714,34.4,24.4,0.3
8,sj,2008,26,2008-06-24,0.0108,0.0499,0.100571,0.117329,8.02,300.601429,...,17.10,77.395714,8.02,17.678571,2.785714,28.657143,6.242857,32.8,23.9,28.7
9,sj,2008,27,2008-07-01,0.07266667,0.10666,0.155429,0.164900,17.52,300.497143,...,11.90,78.534286,17.52,17.808571,2.228571,28.457143,4.628571,31.1,25.0,2.9


In [29]:
yearly_summary=sqlContext.sql("SELECT year,count(*) FROM epidemic_prediction GROUP BY 1 ORDER BY 2 ASC")

View the data

In [11]:
yearly_summary.show(truncate=False)

+----+--------+
|year|count(1)|
+----+--------+
|2008|35      |
|2013|43      |
|2009|51      |
|2010|76      |
|2011|95      |
|2012|101     |
+----+--------+



In [30]:
df2012= sqlContext.sql("SELECT * FROM epidemic_prediction WHERE year = 2012")


In [31]:
pd.DataFrame(df2012.take(10), columns=df2012.columns).head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2012,52,2012-01-01,0.1653,0.099025,0.152875,0.081513,15.30,298.507143,...,8.0,73.187143,15.30,14.714286,2.042857,25.828571,5.785714,29.4,22.2,48.1
1,sj,2012,1,2012-01-08,0.02685,0.0642,0.228514,0.199400,0.00,298.694286,...,1.4,76.517143,0.00,15.550000,2.328571,26.200000,6.114286,30.6,22.2,4.7
2,sj,2012,2,2012-01-15,-0.1263667,0.01268,0.230686,0.124157,0.00,298.204286,...,10.7,78.832857,0.00,15.531429,2.128571,25.414286,6.242857,29.4,21.1,13.0
3,sj,2012,3,2012-01-22,-0.06515,0.0226,0.152140,0.184014,0.00,298.350000,...,1.0,74.550000,0.00,14.798571,2.357143,25.442857,6.000000,29.4,21.7,6.4
4,sj,2012,4,2012-01-29,0.31275,0.1132,0.188986,0.181743,0.00,297.347143,...,13.8,76.894286,0.00,14.365714,2.157143,24.857143,6.028571,28.3,20.6,27.5
5,sj,2012,5,2012-02-05,-0.1114,0.0191,0.141757,0.132314,0.00,297.404286,...,7.9,75.638571,0.00,14.175714,2.471429,24.828571,5.785714,28.3,21.1,32.2
6,sj,2012,6,2012-02-12,-0.0469,0.054,0.255000,0.169086,7.02,297.567143,...,7.8,78.105714,7.02,14.807143,2.314286,24.900000,5.800000,28.3,21.1,25.5
7,sj,2012,7,2012-02-19,-0.0006333333,-0.1169333,0.153657,0.144900,0.00,297.932857,...,1.9,76.967143,0.00,14.934286,2.700000,25.642857,6.828571,31.1,21.7,12.5
8,sj,2012,8,2012-02-26,-0.0569,0.156225,0.385383,0.306250,0.00,297.224286,...,24.5,78.225714,0.00,14.524286,2.528571,24.885714,5.642857,28.3,20.6,8.2
9,sj,2012,9,2012-03-04,-0.01215,0.00042,0.165543,0.163271,0.00,298.032857,...,2.8,75.425714,0.00,14.684286,2.614286,26.142857,6.242857,30.0,21.7,5.8


In [32]:
df2012.count()

101

Create the iceberg named statistics_2012

In [34]:
df2012.writeTo("statistics_2012").using("iceberg").create()

In [35]:
dfsj = sqlContext.sql("SELECT * FROM epidemic_prediction WHERE city = 'iq'")

In [36]:
pd.DataFrame(dfsj.take(10), columns=df2012.columns).head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,iq,2010,26,2010-07-02,0.1837833,0.1425,0.225129,0.150214,82.29,297.648571,...,34.11,92.581429,82.29,17.654286,9.428571,27.440000,10.760000,33.8,21.5,11.2
1,iq,2010,27,2010-07-09,0.2916571,0.2722667,0.330700,0.320914,25.30,298.224286,...,9.10,83.885714,25.30,16.320000,10.157143,27.025000,9.625000,33.0,21.2,8.9
2,iq,2010,28,2010-07-16,0.2085429,0.3664571,0.212629,0.255514,62.14,297.955714,...,61.09,92.057143,62.14,18.030000,9.557143,26.950000,10.350000,33.4,21.6,22.6
3,iq,2010,29,2010-07-23,0.08928571,0.06321429,0.122057,0.081957,47.80,295.715714,...,19.60,88.970000,47.80,15.394286,7.828571,26.900000,9.700000,33.3,14.2,4.8
4,iq,2010,30,2010-07-30,0.3061,0.3276833,0.250086,0.267914,56.30,298.502857,...,18.93,78.610000,56.30,15.468571,11.771429,27.050000,11.850000,33.5,16.9,3.0
5,iq,2010,31,2010-08-06,0.34202,0.3048571,0.332357,0.345729,16.95,298.611429,...,32.92,83.274286,16.95,16.624286,11.157143,27.700000,12.600000,34.0,20.0,92.2
6,iq,2010,32,2010-08-13,0.3322167,0.3848167,0.356600,0.353917,9.88,298.305714,...,10.50,72.471429,9.88,13.990000,11.828571,26.433333,12.033333,34.2,18.5,0.0
7,iq,2010,33,2010-08-20,0.2218143,0.2241,0.273086,0.231243,15.76,298.724286,...,20.30,81.557143,15.76,16.375714,12.642857,27.275000,11.375000,34.0,21.0,20.3
8,iq,2010,35,2010-09-03,0.3420143,0.3019286,0.314700,0.381000,59.06,300.224286,...,41.10,77.202857,59.06,16.801429,12.671429,28.175000,13.200000,36.2,21.0,8.5
9,iq,2010,36,2010-09-10,0.1165833,0.1002833,0.140683,0.113817,37.90,298.501429,...,50.00,85.691429,37.90,17.170000,10.100000,27.100000,11.166667,35.4,21.0,15.0


In [37]:
dfsj.write.format("iceberg").save("statistics_2012",mode='append')

Read the data stored in the iceberg

In [38]:
read_iceberg = spark.read.format("iceberg").load("statistics_2012")

In [39]:
read_iceberg .printSchema()

root
 |-- city: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- week_start_date: string (nullable = true)
 |-- ndvi_ne: string (nullable = true)
 |-- ndvi_nw: string (nullable = true)
 |-- ndvi_se: double (nullable = true)
 |-- ndvi_sw: double (nullable = true)
 |-- precipitation_amt_mm: double (nullable = true)
 |-- reanalysis_air_temp_k: double (nullable = true)
 |-- reanalysis_avg_temp_k: double (nullable = true)
 |-- reanalysis_dew_point_temp_k: double (nullable = true)
 |-- reanalysis_max_air_temp_k: double (nullable = true)
 |-- reanalysis_min_air_temp_k: double (nullable = true)
 |-- reanalysis_precip_amt_kg_per_m2: double (nullable = true)
 |-- reanalysis_relative_humidity_percent: double (nullable = true)
 |-- reanalysis_sat_precip_amt_mm: double (nullable = true)
 |-- reanalysis_specific_humidity_g_per_kg: double (nullable = true)
 |-- reanalysis_tdtr_k: double (nullable = true)
 |-- station_avg_temp_c: double (nul

In [40]:
pd.DataFrame(read_iceberg.take(10), columns=read_iceberg.columns).head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,iq,2010,26,2010-07-02,0.1837833,0.1425,0.225129,0.150214,82.29,297.648571,...,34.11,92.581429,82.29,17.654286,9.428571,27.440000,10.760000,33.8,21.5,11.2
1,iq,2010,27,2010-07-09,0.2916571,0.2722667,0.330700,0.320914,25.30,298.224286,...,9.10,83.885714,25.30,16.320000,10.157143,27.025000,9.625000,33.0,21.2,8.9
2,iq,2010,28,2010-07-16,0.2085429,0.3664571,0.212629,0.255514,62.14,297.955714,...,61.09,92.057143,62.14,18.030000,9.557143,26.950000,10.350000,33.4,21.6,22.6
3,iq,2010,29,2010-07-23,0.08928571,0.06321429,0.122057,0.081957,47.80,295.715714,...,19.60,88.970000,47.80,15.394286,7.828571,26.900000,9.700000,33.3,14.2,4.8
4,iq,2010,30,2010-07-30,0.3061,0.3276833,0.250086,0.267914,56.30,298.502857,...,18.93,78.610000,56.30,15.468571,11.771429,27.050000,11.850000,33.5,16.9,3.0
5,iq,2010,31,2010-08-06,0.34202,0.3048571,0.332357,0.345729,16.95,298.611429,...,32.92,83.274286,16.95,16.624286,11.157143,27.700000,12.600000,34.0,20.0,92.2
6,iq,2010,32,2010-08-13,0.3322167,0.3848167,0.356600,0.353917,9.88,298.305714,...,10.50,72.471429,9.88,13.990000,11.828571,26.433333,12.033333,34.2,18.5,0.0
7,iq,2010,33,2010-08-20,0.2218143,0.2241,0.273086,0.231243,15.76,298.724286,...,20.30,81.557143,15.76,16.375714,12.642857,27.275000,11.375000,34.0,21.0,20.3
8,iq,2010,35,2010-09-03,0.3420143,0.3019286,0.314700,0.381000,59.06,300.224286,...,41.10,77.202857,59.06,16.801429,12.671429,28.175000,13.200000,36.2,21.0,8.5
9,iq,2010,36,2010-09-10,0.1165833,0.1002833,0.140683,0.113817,37.90,298.501429,...,50.00,85.691429,37.90,17.170000,10.100000,27.100000,11.166667,35.4,21.0,15.0
